# Create Gold Standard Wikidata National Parks

In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from wd_species import *
import pprint
import random
from sklearn.utils.extmath import cartesian
from similarity.jarowinkler import JaroWinkler

In [2]:

def createDFfromXML_list(path):

    # create list of all columns from target schema
    path_target = '../xml/Final_schema_XML.xml'
    tree_target = ET.parse(path_target)
    root_target = tree_target.getroot()
    rows_target = []
    columns = []
    for species_target in root_target.iter('Species'):
        for attr_target in species_target:
            if attr_target not in columns:
                columns.append(attr_target.tag)


    # read in xml and create df
    tree = ET.parse(path)
    root = tree.getroot()
    rows = []
    for species in root.iter('Species'):

        col_dict = {}
        for col in columns:

            for attr in species.iter(col):
                if col in ['ID', 'Provenance']:
                    col_dict[col] = attr.text

                else:
                    temp_list = []
                    for child in attr:
                        temp_list.append(child.text)

                    if len(temp_list) == 0:
                        pass
                    elif len(temp_list) == 1:
                        col_dict[col] = temp_list[0]
                    else:
                        col_dict[col] = temp_list


        rows.append(col_dict)

    df = pd.DataFrame(rows, columns=columns)        
    return df

In [3]:

def createDFfromXML(path):

    # create list of all columns from target schema
    path_target = '../xml/Final_schema_XML.xml'
    tree_target = ET.parse(path_target)
    root_target = tree_target.getroot()
    rows_target = []
    columns = []
    for species_target in root_target.iter('Species'):
        for attr_target in species_target:
            if attr_target not in columns:
                columns.append(attr_target.tag)

    
    # To be deleted afterwards!
    columns.append('Scientific_Name')
    
    
    # read in xml and create df
    tree = ET.parse(path)
    root = tree.getroot()
    rows = []
    for species in root.iter('Species'):

        col_dict = {}
        for col in columns:

            for attr in species.iter(col):
                if col in ['ID', 'Provenance', 'Scientific_Name']:
                    col_dict[col] = attr.text

                else:
                    temp_list = []
                    for child in attr:
                        temp_list.append(child.text)

                    if len(temp_list) == 0:
                        pass
                    elif len(temp_list) == 1:
                        col_dict[col] = temp_list[0]
                    else:
                        col_dict[col] = temp_list


        rows.append(col_dict)

    df = pd.DataFrame(rows, columns=columns)        
    return df

In [4]:
dataset_dict = {}
dataset_dict['wd'] = {}   #WikiData
dataset_dict['es'] = {}   #EndangeroudSpecies
dataset_dict['np'] = {}   #NationalParks

dataset_dict['wd']['path'] = getPathXML()   #WikiData
dataset_dict['es']['path'] = '../xml/endangered_species.xml'   #EndangeroudSpecies
dataset_dict['np']['path'] = '../xml/biodiversity.xml'   #NationalParks


for dataset in dataset_dict.keys():
    # read in xmls
    print(dataset)
    
    #if dataset == 'es':
    #else:
    #    dataset_dict[dataset]['df'] = createDFfromXML_list(dataset_dict[dataset]['path'])
        
    dataset_dict[dataset]['df'] = createDFfromXML(dataset_dict[dataset]['path'])
    
    
    
#dataset_dict

wd
es
np


In [5]:
# to be deleted afterwards
'''display(dataset_dict['es']['df'].head(2))
display(dataset_dict['np']['df'].head(2))

dataset_dict['es']['df']['Scientific_Names'] = dataset_dict['es']['df']['Scientific_Name']
dataset_dict['es']['df'] = dataset_dict['es']['df'].drop(columns = 'Scientific_Name')

display(dataset_dict['es']['df'].head(2))'''

"display(dataset_dict['es']['df'].head(2))\ndisplay(dataset_dict['np']['df'].head(2))\n\ndataset_dict['es']['df']['Scientific_Names'] = dataset_dict['es']['df']['Scientific_Name']\ndataset_dict['es']['df'] = dataset_dict['es']['df'].drop(columns = 'Scientific_Name')\n\ndisplay(dataset_dict['es']['df'].head(2))"

### Search for categories 

In [6]:
# retrieve categories
for dataset in dataset_dict.keys():
    categories = dataset_dict[dataset]['df'].Categories
    # circumvent break because of lists in wd
    categories = categories.apply(lambda x: str(x)) 
    dataset_dict[dataset]['categories'] = categories.unique()

In [7]:
# exploration
for dataset in dataset_dict.keys():
    print(dataset)
    print(dataset_dict[dataset]['df'].info(), '\n\n')

wd
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ID                100000 non-null  object 
 1   Provenance        100000 non-null  object 
 2   Scientific_Name   99999 non-null   object 
 3   Common_Names      100000 non-null  object 
 4   Labels            0 non-null       float64
 5   Where_Listed_l    0 non-null       float64
 6   Different_From_l  346 non-null     object 
 7   Endemic_To_l      3960 non-null    object 
 8   Regions           0 non-null       float64
 9   Region_Names      0 non-null       float64
 10  Listing_Statuses  26637 non-null   object 
 11  Categories        90563 non-null   object 
 12  Orders            99403 non-null   object 
 13  Families          99738 non-null   object 
 14  States            0 non-null       float64
 15  Scientific_Name   99999 non-null   object 
dtypes: float64(5), obj

In [8]:

# create blank of mapping dict
for dataset in dataset_dict.keys():
    dataset_dict[dataset]['mapping'] = {}
    for categ in dataset_dict[dataset]['categories']:
        dataset_dict[dataset]['mapping'][categ] = ''
    
    #if dataset != 'wd':
    print(dataset)
    pprint.pprint(dataset_dict[dataset]['mapping'])

wd
{"['adenophorea', 'chromadorea']": '',
 "['agaricomycetes', 'pucciniomycetes']": '',
 "['amphibia', 'dipnotetrapodomorpha']": '',
 "['arachnida', 'merostomata', 'xiphosura']": '',
 "['bird', 'dipnotetrapodomorpha', 'reptilia']": '',
 "['bird', 'reptilia', 'dipnotetrapodomorpha']": '',
 "['bivalvia', 'gastropoda']": '',
 "['bryopsida', 'equisetopsida']": '',
 "['cephalaspidomorphi', 'petromyzonti', 'petromyzontida']": '',
 "['cephalaspidomorphi', 'petromyzontida', 'petromyzonti']": '',
 "['chondrichthyes', 'elasmobranchii']": '',
 "['chondrichthyes', 'holocephali']": '',
 "['choripetalae', 'dicotyledones', 'rosopsida', 'magnoliopsida']": '',
 "['choripetalae', 'magnoliopsida', 'rosopsida', 'dicotyledones']": '',
 "['chromadorea', 'adenophorea']": '',
 "['cladistei', 'fish', 'cladistii']": '',
 "['cladistii', 'fish', 'cladistei']": '',
 "['clitellata', 'oligochaeta']": '',
 "['coniferopsida', 'equisetopsida', 'pinopsida']": '',
 "['coniferopsida', 'pinopsida', 'equisetopsida']": '',
 

In [9]:
dataset_dict['es']['mapping'] = {
     'Amphibians': 8,
     'Arachnids': 7,
     'Birds': 2,
     'Clams': 6,   #?
     'Conifers and Cycads': 11,
     'Crustaceans': 9,
     'Ferns and Allies': 11,
     'Fishes': 4,
     'Flowering Plants': 11,
     'Insects': 3,
     'Lichens': 10,  #?
     'Mammals': 1,
     'Reptiles': 5,
     'Snails': 6}



dataset_dict['np']['mapping'] = {
     'Algae': 11,
     'Amphibian': 8,
     'Bird': 2,
     'Crab/Lobster/Shrimp': 9,
     'Fish': 4,
     'Fungi': 10, #?
     'Insect': 3,
     'Invertebrate': 6,
     'Mammal': 1,
     'Nonvascular Plant': 11,
     'Reptile': 5,
     'Slug/Snail': 6,
     'Spider/Scorpion': 7,
     'Vascular Plant': 11}


### Assign mappings

In [10]:
for dataset in dataset_dict.keys():
    if dataset != 'wd':
        dataset_dict[dataset]['df']['mapping'] = dataset_dict[dataset]['df']['Categories'].apply(lambda x: dataset_dict[dataset]['mapping'][x])

In [11]:

mapping_lidentifiers = np.sort(dataset_dict['np']['df']['mapping'].unique())
mapping_lidentifiers

for dataset in dataset_dict.keys():
    if dataset != 'wd':
        dataset_dict[dataset]['df_perCat'] = []
        
        for mapping_nr in mapping_lidentifiers:
            df_perCat = dataset_dict[dataset]['df'][dataset_dict[dataset]['df']['mapping'] == mapping_nr]
            dataset_dict[dataset]['df_perCat'].append(df_perCat)
        



In [12]:
# visual check of datasets
for dataset in dataset_dict.keys():
    if dataset != 'wd':
        
        for df in dataset_dict[dataset]['df_perCat']:
            display(df.head(2))
            
        break

,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
60,ES1004,FWSEndangeredSpecies,Antilocapra americana sonoriensis,Sonoran pronghorn,NaN,NaN,NaN,NaN,2,Southwest,NaN,Mammals,NaN,NaN,NaN,Antilocapra americana sonoriensis,1
65,ES974,FWSEndangeredSpecies,Aplodontia rufa nigra,Point Arena mountain beaver,NaN,NaN,NaN,NaN,8,Pacific Southwest,NaN,Mammals,NaN,NaN,NaN,Aplodontia rufa nigra,1


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
7,ES1512,FWSEndangeredSpecies,Accipiter striatus venator,Puerto Rican sharp-shinned hawk,NaN,NaN,NaN,NaN,4,Southeast,NaN,Birds,NaN,NaN,NaN,Accipiter striatus venator,2
15,ES1589,FWSEndangeredSpecies,Acrocephalus familiaris kingi,Nihoa millerbird (old world warbler),NaN,NaN,NaN,NaN,1,Pacific,NaN,Birds,NaN,NaN,NaN,Acrocephalus familiaris kingi,2


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
39,ES1120,FWSEndangeredSpecies,Ambrysus amargosus,Ash Meadows naucorid,NaN,NaN,NaN,NaN,8,Pacific Southwest,NaN,Insects,NaN,NaN,NaN,Ambrysus amargosus,3
53,ES999,FWSEndangeredSpecies,Anaea troglodyta floridalis,Florida leafwing Butterfly,NaN,NaN,NaN,NaN,4,Southeast,NaN,Insects,NaN,NaN,NaN,Anaea troglodyta floridalis,3


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
11,ES1109,FWSEndangeredSpecies,Acipenser oxyrinchus (=oxyrhynchus) desotoi,Gulf sturgeon,NaN,NaN,NaN,NaN,4,Southeast,NaN,Fishes,NaN,NaN,NaN,Acipenser oxyrinchus (=oxyrhynchus) desotoi,4
12,ES1476,FWSEndangeredSpecies,Acipenser transmontanus,White sturgeon,NaN,NaN,NaN,NaN,1,Pacific,NaN,Fishes,NaN,NaN,NaN,Acipenser transmontanus,4


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
45,ES1371,FWSEndangeredSpecies,Ameiva polops,St. Croix ground lizard,NaN,NaN,NaN,NaN,4,Southeast,NaN,Reptiles,NaN,NaN,NaN,Ameiva polops,5
59,ES1036,FWSEndangeredSpecies,Anolis roosevelti,Culebra Island giant anole,NaN,NaN,NaN,NaN,4,Southeast,NaN,Reptiles,NaN,NaN,NaN,Anolis roosevelti,5


...

,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
17,ES1323,FWSEndangeredSpecies,Adelocosa anops,Kauai cave wolf or pe'e pe'e maka 'ole spider,NaN,NaN,NaN,NaN,1,Pacific,NaN,Arachnids,NaN,NaN,NaN,Adelocosa anops,7
253,ES1522,FWSEndangeredSpecies,Cicurina baronia,Robber Baron Cave Meshweaver,NaN,NaN,NaN,NaN,2,Southwest,NaN,Arachnids,NaN,NaN,NaN,Cicurina baronia,7


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
40,ES1552,FWSEndangeredSpecies,Ambystoma bishopi,Reticulated flatwoods salamander,NaN,NaN,NaN,NaN,4,Southeast,NaN,Amphibians,NaN,NaN,NaN,Ambystoma bishopi,8
41,ES1015,FWSEndangeredSpecies,Ambystoma californiense,California tiger Salamander,NaN,NaN,NaN,NaN,0,Pacific Southwest,NaN,Amphibians,NaN,NaN,NaN,Ambystoma californiense,8


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
62,ES1199,FWSEndangeredSpecies,Antrolana lira,Madison Cave isopod,NaN,NaN,NaN,NaN,5,Northeast,NaN,Crustaceans,NaN,NaN,NaN,Antrolana lira,9
155,ES1623,FWSEndangeredSpecies,Branchinecta conservatio,Conservancy fairy shrimp,NaN,NaN,NaN,NaN,8,Pacific Southwest,NaN,Crustaceans,NaN,NaN,NaN,Branchinecta conservatio,9


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
263,ES815,FWSEndangeredSpecies,Cladonia perforata,Florida perforate cladonia,NaN,NaN,NaN,NaN,4,Southeast,NaN,Lichens,NaN,NaN,NaN,Cladonia perforata,10
695,ES277,FWSEndangeredSpecies,Gymnoderma lineare,Rock gnome lichen,NaN,NaN,NaN,NaN,4,Southeast,NaN,Lichens,NaN,NaN,NaN,Gymnoderma lineare,10


,ID,Provenance,Scientific_Name,Common_Names,Labels,Where_Listed_l,Different_From_l,Endemic_To_l,Regions,Region_Names,Listing_Statuses,Categories,Orders,Families,States,Scientific_Name,mapping
0,ES570,FWSEndangeredSpecies,Abronia macrocarpa,Large-fruited sand-verbena,NaN,NaN,NaN,NaN,2,Southwest,NaN,Flowering Plants,NaN,NaN,NaN,Abronia macrocarpa,11
1,ES458,FWSEndangeredSpecies,Abutilon eremitopetalum,No common name,NaN,NaN,NaN,NaN,1,Pacific,NaN,Flowering Plants,NaN,NaN,NaN,Abutilon eremitopetalum,11


In [13]:
scientificNameColumnLabel = 'Scientific_Names'

def getScientificNameFromID(dataframe, id_species):
    
    try:
        row       = dataframe[dataframe['ID'] == id_species]
        index     = row.index[0]
        value     = row.at[index, scientificNameColumnLabel]

        # handle nan values
        if type(value) != str:
            value = ''
    except:
        value = ''
    
    return value

### Create template for gold standard for NP (biodiversity) and endangered species

In [ ]:
%%time
ds1 = 'np'
ds2 = 'es'

jarowinkler = JaroWinkler()

columns_scoreDF = ['id1', 'id2', 'match', 'jarowinkler', 'potentiallyTrue']
final_df=pd.DataFrame(columns=columns_scoreDF)

comparisons_per_df = []

full_id2_list = np.array(dataset_dict[ds2]['df']['ID'])


for mapping_nr in mapping_lidentifiers:

    matches = []
    
    # assignments
    dataframe1 = dataset_dict[ds1]['df_perCat'][mapping_nr-1].copy()
    dataframe2 = dataset_dict[ds2]['df_perCat'][mapping_nr-1].copy()
    
    id_ds1 = np.array(dataframe1['ID'])
    id_ds2 = np.array(dataframe2['ID']) # delete
     
    print(mapping_nr, len(id_ds1), len(id_ds2))
    
    for current_id in id_ds1:
        
        
        sciName_of_current_id = getScientificNameFromID(dataframe1, current_id)
        dataframe2[scientificNameColumnLabel+'_short'] = dataframe2[scientificNameColumnLabel].apply(lambda x: x[0:3].lower())
        possible_matches_df = dataframe2[dataframe2[scientificNameColumnLabel+'_short'] == sciName_of_current_id[0:3].lower()]
        id_ds2 = list(possible_matches_df['ID'])
        matches.append(len(id_ds2))
    
        if len(id_ds2) > 0:
            
            temp_df = pd.DataFrame(columns = columns_scoreDF)
            
            # create cartesian product for this category
            cart_array = cartesian(([current_id], id_ds2))

            # calulate jaro winkler
            jarowinkl_result = [jarowinkler.similarity(sciName_of_current_id, getScientificNameFromID(dataframe2, x[1])) for x in cart_array]
            jarowinkl_result = np.array(jarowinkl_result)

            temp_df['id1']             = cart_array[:,0]
            temp_df['id2']             = cart_array[:,1]
            temp_df['match']           = ''
            temp_df['jarowinkler']     = jarowinkl_result
            temp_df['potentiallyTrue'] = True

            # find combination with highest score for each id from id_list 1
        
            current_df = temp_df[temp_df.id1 == current_id]
            entry = current_df[current_df.jarowinkler == np.max(current_df.jarowinkler)]
            final_df   = pd.concat([final_df, entry])
        
        #for all ather cases
        
        else: 
            data = {
                'id1':           current_id, 
                'id2':           full_id2_list[random.randint(0, len(full_id2_list)-1)], 
                'match':         '',
                'jarowinkler':   0,
                'potentiallyTrue': False
            }
            entry = pd.DataFrame(data, columns=columns_scoreDF, index=[0])
        
        final_df   = pd.concat([final_df, entry])
        
    comparisons_per_df.append(sum(matches))
    final_df.to_csv('../Data/goldstandard/intermediates/goldstandardTemplate.csv')


print('total comparisons: ', sum(comparisons_per_df))



In [ ]:
display(final_df.shape)
final_df.head(3)

In [ ]:
# join both datasets to final df
leftjoinedDF             = pd.merge(left=final_df,     right=dataset_dict[ds1]['df'], how='left', left_on='id1', right_on='ID')
match_table_goldstandard = pd.merge(left=leftjoinedDF, right=dataset_dict[ds2]['df'], how='left', left_on='id2', right_on='ID')
match_table_goldstandard.to_csv(  '../Data/goldstandard/intermediates/match_table_goldstandard_BIO_ES.csv',  index=False)
match_table_goldstandard.to_excel('../Data/goldstandard/intermediates/match_table_goldstandard_BIO_ES.xlsx', index=False)
match_table_goldstandard


### Create template for gold standard for NP (biodiversity) and WD

In [ ]:
%%time
ds1 = 'np'
ds2 = 'wd'

jarowinkler = JaroWinkler()

columns_scoreDF = ['id1', 'id2', 'match', 'jarowinkler', 'potentiallyTrue']
final_df=pd.DataFrame(columns=columns_scoreDF)

comparisons_per_df = []

full_id2_list = np.array(dataset_dict[ds2]['df']['ID'])


matches = []
lenght_of_string_to_match = 3
count = 0

def createShortForms(elem):
    try:
        returnvalue = elem[0:lenght_of_string_to_match].lower()
    except:
        returnvalue = ''
        
    return returnvalue

# assignments
dataframe1 = dataset_dict[ds1]['df'].copy()
dataframe2 = dataset_dict[ds2]['df'].copy()

id_ds1 = np.array(dataframe1['ID'])
id_ds2 = np.array(dataframe2['ID']) # delete

print(len(id_ds1), len(id_ds2))

for current_id in id_ds1:

    count += 1
    if count%2500 == 0:
        print(count)
        
    sciName_of_current_id = getScientificNameFromID(dataframe1, current_id)
    dataframe2[scientificNameColumnLabel+'_short'] = dataframe2[scientificNameColumnLabel].apply(lambda x: createShortForms(x))
    possible_matches_df = dataframe2[dataframe2[scientificNameColumnLabel+'_short'] == sciName_of_current_id[0:lenght_of_string_to_match].lower()]
    id_ds2 = list(possible_matches_df['ID'])
    matches.append(len(id_ds2))

    if len(id_ds2) > 0:

        temp_df = pd.DataFrame(columns = columns_scoreDF)

        # create cartesian product for this category
        cart_array = cartesian(([current_id], id_ds2))

        # calulate jaro winkler
        jarowinkl_result = [jarowinkler.similarity(sciName_of_current_id, getScientificNameFromID(dataframe2, x[1])) for x in cart_array]
        jarowinkl_result = np.array(jarowinkl_result)

        temp_df['id1']             = cart_array[:,0]
        temp_df['id2']             = cart_array[:,1]
        temp_df['match']           = ''
        temp_df['jarowinkler']     = jarowinkl_result
        temp_df['potentiallyTrue'] = True

        # find combination with highest score for each id from id_list 1

        current_df = temp_df[temp_df.id1 == current_id]
        entry = current_df[current_df.jarowinkler == np.max(current_df.jarowinkler)]
        final_df   = pd.concat([final_df, entry])

    #for all ather cases

    else: 
        data = {
            'id1':           current_id, 
            'id2':           full_id2_list[random.randint(0, len(full_id2_list)-1)], 
            'match':         '',
            'jarowinkler':   0,
            'potentiallyTrue': False
        }
        entry = pd.DataFrame(data, columns=columns_scoreDF, index=[0])

    final_df   = pd.concat([final_df, entry])

comparisons_per_df.append(sum(matches))
final_df.to_csv('../Data/goldstandard/intermediates/goldstandardTemplate_NP_WD.csv')


print('total comparisons: ', sum(comparisons_per_df))




In [ ]:
final_df.to_csv('../Data/goldstandard/intermediates/goldstandardTemplate_NP_WD.csv')
final_df